#Day 4: Transfer Learning (The Pro Move)
Welcome to Day 4!

On Day 3, we built a "pro-level" CNN that was stable, highly accurate (~98-99%), and didn't overfit. We are now very good at building and training a network from scratch.

But what if we didn't have to?

Today's Goal: We will learn Transfer Learning, the most powerful and common technique used in modern computer vision.

Instead of building a "baby" network that has never seen an image, we will download a "pro" network (like VGG16) that has already been trained by experts at Google or Oxford on millions of images.

Today's Plan:

Load Data: Prepare our fruit dataset one last time.

Theory: What is Transfer Learning?

Load a Pre-trained Model (VGG16): We'll download the VGG16 model, pre-trained on the "ImageNet" dataset.

"Freeze" the Base: We'll lock the original VGG16 layers so they don't change.

Build Our "Head": We'll add our own Dense layers on top of VGG16 to solve our specific fruit problem.

Train (Feature Extraction): We'll train only our new top layers.

Theory: What is Fine-Tuning?

"Unfreeze" and Fine-Tune: We'll unlock a few of the top VGG16 layers and re-train them with a tiny learning rate to get even better results.

Compare: We'll compare our Day 2, Day 3, and Day 4 models to see the incredible power of this technique.

##Cell 1: Setup - Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import math
from tensorflow.keras import layers
from tensorflow.keras import regularizers

print(f"TensorFlow Version: {tf.__version__}")
print("All libraries imported.")

##Cell 2: Define Dataset Parameters
This time, we must change IMG_SIZE to 224.

In [ ]:
# We will resize all images to 224x224
IMG_SIZE = 224
print(f"Image size set to: {IMG_SIZE}x{IMG_SIZE} pixels")

###Learning Note: Why IMG_SIZE = 224?

The VGG16 model we are about to use was pre-trained on the ImageNet dataset, where all images were 224x224 pixels.

We must use the same image size that the model was originally trained on. If we gave it 64x64 images, the internal dimensions would not match, and the model would fail.

In [ ]:
# We can use a smaller batch size since the images are much larger
BATCH_SIZE = 16
print(f"Batch size set to: {BATCH_SIZE}")

# Our images have 3 color channels (R, G, B)
CHANNELS = 3
print(f"Number of color channels: {CHANNELS}")

# Our dataset has 6 classes
NUM_CLASSES = 6
print(f"Number of classes: {NUM_CLASSES}")

##Cell 3: Define File Paths

In [ ]:
# ===============================================
# Ececute this cell only when you have Dataset already downloaded else , go to next cell
# =============================================

import os

# Base directory
base_dir = r"D:\dataset"  #set your dataset path
print(f"Base directory: {base_dir}")
print("Exists:", os.path.exists(base_dir))

# Training data path
train_dir = os.path.join(base_dir, "train")
print(f"Training data path: {train_dir}")
print("Exists:", os.path.exists(train_dir))

# Test data path
test_dir = os.path.join(base_dir, "test")
print(f"Test data path: {test_dir}")
print("Exists:", os.path.exists(test_dir))


In [ ]:
# =============================================
# Execute this code only when you Dont have dataset || prefer when you are executing this in Google colab
# ==============================================
# Download the dataset from Kaggle using kagglehub

import os
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")

print("Path to dataset files:", path)

# The downloaded dataset structure is typically dataset/train and dataset/test
# Adjust the base_dir to point to the extracted dataset folder
base_dir = os.path.join(path, "dataset")

print(f"Base directory: {base_dir}")
print("Exists:", os.path.exists(base_dir))


# Training data path
train_dir = os.path.join(base_dir, "train")
print(f"Training data path: {train_dir}")
print("Exists:", os.path.exists(train_dir))

# Test data path
test_dir = os.path.join(base_dir, "test")
print(f"Test data path: {test_dir}")
print("Exists:", os.path.exists(test_dir))

##Cell 4: Load and Prepare Datasets

In [ ]:
# Load Training Data
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)
print("Loaded Training Data.")

# Load Validation/Test Data
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)
print("Loaded Validation (Test) Data.")

# Get Class Names
class_names = train_dataset.class_names
print(f"Class names: {class_names}")

# Configure for Performance
AUTOTUNE = tf.data.AUTOTUNE
# We add .repeat() to make sure the dataset loops for each epoch
train_ds = train_dataset.cache().repeat().prefetch(buffer_size=AUTOTUNE)
val_ds = validation_dataset.cache().repeat().prefetch(buffer_size=AUTOTUNE)
print("Applied .cache(), .repeat(), and .prefetch() to both datasets.")

In [ ]:
# Calculate steps_per_epoch for training
# (We know these numbers from the output of the cell above)
steps_per_epoch = math.ceil(10901 / BATCH_SIZE)

# Calculate validation_steps for testing
validation_steps = math.ceil(2698 / BATCH_SIZE)

print(f"Total training steps per epoch: {steps_per_epoch}")
print(f"Total validation steps per epoch: {validation_steps}")

##Cell 5: What is Transfer Learning? (Theory)
Transfer Learning is the process of using a model that was pre-trained on a large, general dataset (like ImageNet, which has 1.2 million images of 1000 classes like "dog," "car," "cat," etc.) and adapting it to solve a new, specific task (like our fruit problem).

Why do this?

The pre-trained model (like VGG16) has already spent thousands of hours of GPU time learning to "see."

Its first layers already know how to find edges and corners.

Its middle layers know how to find textures and shapes.

Its upper layers know how to find complex objects (like fur, wheels, or eyes).

We don't need to re-teach this! We can just "transfer" all that knowledge.

Our Two-Step Process:

Feature Extraction (Training): We will "chop off" the original top layer of VGG16 (which predicted 1000 classes) and "freeze" the entire base. We then add our own small Dense layer "head" (which predicts 6 fruit classes) and train only our new head.

Fine-Tuning (Optimizing): After our new head is trained, we will "unfreeze" a few of the top layers from the VGG16 base and continue training everything with a very low learning rate. This allows the VGG model to "fine-tune" its high-level feature detectors (e.g., "fur") to become better at detecting our specific features (e.g., "rotten spots").

##Cell 6: Load the Pre-Trained VGG16 Base
Let's load the VGG16 model from Keras. We'll pass three important arguments:

weights='imagenet': This automatically downloads the weights it learned from the ImageNet dataset.

include_top=False: This is the key. It means "don't include the final 1000-neuron Dense layer." We're "chopping off the head."

input_shape=(...): We tell the model what size our images are.

In [ ]:
from tensorflow.keras.applications import VGG16

# Load the VGG16 model, pre-trained on ImageNet, without its top classifier
base_model_vgg = VGG16(
    weights='imagenet',
    include_top=False,  # This is the key: we want to build our *own* classifier head
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

print("VGG16 base model loaded.")

##Cell 7: Freeze the VGG16 Base
This is a critical step. We need to tell Keras, "Do NOT update the weights of the VGG16 layers during our training." We want to keep all the knowledge it already has.

In [ ]:
# Set the base model's layers to be non-trainable
base_model_vgg.trainable = False
print("VGG16 base model is now 'frozen' (non-trainable).")

In [ ]:
# Let's look at the summary
print("\n--- VGG16 Summary ---")
base_model_vgg.summary()

###Analysis:

Look at that! This base model has 14.7 Million parameters (all dedicated to finding features).

And at the bottom, it shows: Trainable params: 0. This confirms our model is "frozen."

##Cell 8: Build Our New Model (Model 4)
Now, we'll build our new model using the Functional API.

Input: Our (224, 224, 3) image.

Preprocessing: We need to use a special preprocess_input function that VGG16 expects. This normalizes the pixels in the exact same way the model was originally trained (e.g., scaling pixel values to -1 to 1 instead of 0 to 1).

VGG16 Base: We'll pass the preprocessed image to our frozen base_model_vgg.

Our "Head": We'll add a GlobalAveragePooling2D layer, a Dense layer, and our Dropout layer, just like our "Pro" model from Day 3.

Output: Our final 6-class softmax layer.

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Lambda # <-- Make sure to import this

# --- Build the Transfer Learning Model (Functional API) ---

# 1. Input Layer
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input_image')

# 2. Preprocessing Layer
# We wrap the function in a Lambda layer to make it part of the model
x = Lambda(preprocess_input, name='vgg_preprocessing')(inputs)

# 3. VGG16 Base
# We run the frozen base model in "inference mode" (training=False)
# THE FIX: We remove name='vgg16_base' from this call.
x = base_model_vgg(x, training=False)

# 4. Our New "Head" (similar to Day 3)
x = layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
x = layers.Dense(128, activation='relu', name='dense_head')(x)
x = layers.Dropout(0.4, name='dropout_head')(x)

# 5. Output Layer
outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='output_layer')(x)

# Create the final model
model_vgg = keras.Model(inputs=inputs, outputs=outputs)

print("Transfer Learning model with VGG16 base built successfully.")

##Cell 9: Model 4 Summary
Let's look at the summary for our new combined model.

In [ ]:
model_vgg.summary()

###Analysis: The Best of Both Worlds!

Look at the parameters at the bottom:

Total params: 14,780,294 (14.7 Million!)

Trainable params: 66,310 (Only 66 thousand!)

Non-trainable params: 14,713,984

This is perfect! The 14.7M "frozen" parameters from VGG16 will do all the heavy lifting of "seeing," and we only have to train our tiny 66k parameter "head" to do the final classification.

##Cell 10: Compile the Model 4

In [ ]:
model_vgg.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("VGG Transfer Learning model compiled.")

##Cell 11: Train the Model (Feature Extraction)
Let's train our new model. We will only train for 5 epochs.

Watch the val_accuracy. Because the VGG16 base is so powerful, we should see extremely high accuracy almost immediately.

In [ ]:
print("Starting VGG model training (Feature Extraction) for 5 epochs...")
# We save the history to plot it
history_vgg = model_vgg.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    verbose=1 # Show progress
)

print("\nVGG training complete.")

##Cell 12: Visualize VGG Training History (Plots)

In [ ]:
# Convert the new history to a DataFrame
history_vgg_df = pd.DataFrame(history_vgg.history)

# Plot Loss
history_vgg_df[['loss', 'val_loss']].plot(figsize=(10, 6))
plt.title("VGG Transfer Learning: Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

# Plot Accuracy
history_vgg_df[['accuracy', 'val_accuracy']].plot(figsize=(10, 6))
plt.title("VGG Transfer Learning: Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()

##Cell 13: Analyze the Results
This is incredible!

Look at your plots. The val_accuracy (orange) line probably started at ~95% or higher and may have ended around 98-99%... in only 5 epochs!

This is the power of Transfer Learning. We didn't have to teach the model what an "edge" or "texture" is. We just taught our tiny 66k-parameter "head" how to connect VGG16's advanced features to our 6 fruit classes.

###Now, let's try to get that last 1-2% with Fine-Tuning.

##Cell 14: Save the VGG Feature-Extraction Model

In [ ]:
# --- Define file name ---
vgg_model_path = "day4_vgg_model.keras"

# --- Save the VGG Feature Extraction Model ---
try:
    print(f"\nSaving VGG model to: {vgg_model_path} ...")
    model_vgg.save(vgg_model_path)
    print("VGG model saved successfully.")
except Exception as e:
    print(f"An error occurred while saving model_vgg: {e}")

##Cell 15: What is "Fine-Tuning"? (Theory)
Fine-Tuning is the optional second step of transfer learning.

First, we did Feature Extraction (Cells 11-13). We "froze" 100% of the VGG16 base and trained our new "head."

Now, we do Fine-Tuning. We will "unfreeze" the last few layers of the VGG16 base and continue training everything (the unfrozen base layers + our head) with a very, very low learning rate.

Why do this?

The last few layers of VGG16 are specialized for "ImageNet" (e.g., they know "fur" and "feathers"). By unfreezing them, we allow our optimizer to "fine-tune" them to be better at our task (e.g., "rotten spots" and "banana curves").

###Why the low learning rate?

We must use a very low learning rate (like 1e-5). If we use a high rate like adam's default (1e-3), the optimizer will make huge, destructive changes to the carefully pre-trained VGG weights, ruining all the knowledge. We just want to "nudge" them, not rewrite them.

##Cell 16: Unfreeze the Top Layers
First, let's "unfreeze" the VGG base so we can change its weights.

In [ ]:
# Unfreeze the base
base_model_vgg.trainable = True

# Let's see how many layers are in the base
print(f"There are {len(base_model_vgg.layers)} layers in the VGG16 base model.")

In [ ]:
# We will freeze all layers EXCEPT the last 4
# This means we will fine-tune 'block5_conv3' and 'block5_pool'
fine_tune_at = -4

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model_vgg.layers[:fine_tune_at]:
    layer.trainable = False

print("Unfroze the top 4 layers of VGG16 for fine-tuning.")

##Cell 17: Compile for Fine-Tuning
This is the most critical step of fine-tuning. We must re-compile the model, this time with a very low learning rate.

In [ ]:
# We use the Adam optimizer but with a much lower learning rate
optimizer_fine_tune = tf.keras.optimizers.Adam(learning_rate=1e-5)

model_vgg.compile(
    optimizer=optimizer_fine_tune,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model re-compiled for fine-tuning with a low learning rate (1e-5).")

In [ ]:
# Let's check the summary again
print("\n--- Model Summary (for Fine-Tuning) ---")
model_vgg.summary()

###Analysis:

Look at the parameters now!

Total params: 14,780,294

Trainable params: 7,149,062 (7.1 Million!)

Non-trainable params: 7,631,232

We are now re-training our 66k head PLUS over 7 million parameters from the top of the VGG16 model.

##Cell 18: Train the Model (Fine-Tuning)
Let's continue training our model for 10 more epochs. We will use the initial_epoch=5 argument to tell the model to "start counting" from epoch 5, which is where our first training run left off.

In [ ]:
# We set initial_epoch to continue where we left off
initial_epoch = 5
fine_tune_epochs = 10
total_epochs = initial_epoch + fine_tune_epochs

print(f"Starting VGG model fine-tuning for {fine_tune_epochs} more epochs...")
# We'll save the history again
history_vgg_fine_tune = model_vgg.fit(
    train_ds,
    epochs=total_epochs,
    initial_epoch=initial_epoch,  # Start counting from here
    validation_data=val_ds,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    verbose=1
)

print("\nVGG fine-tuning complete.")

##Cell 19: Visualize Fine-Tuning History
Let's plot both training sessions on one graph to see the full story.

In [ ]:
# Combine the history from the first training run with the fine-tuning run

# Get the first 5 epochs from the feature extraction run
acc = history_vgg.history['accuracy']
val_acc = history_vgg.history['val_accuracy']
loss = history_vgg.history['loss']
val_loss = history_vgg.history['val_loss']

# Add the 10 epochs from the fine-tuning run
acc.extend(history_vgg_fine_tune.history['accuracy'])
val_acc.extend(history_vgg_fine_tune.history['val_accuracy'])
loss.extend(history_vgg_fine_tune.history['loss'])
val_loss.extend(history_vgg_fine_tune.history['val_loss'])

# Plot the combined accuracy
plt.figure(figsize=(10, 6))
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.axvline(initial_epoch - 1, color='red', linestyle='--', label='Start Fine-Tuning') # Add a line
plt.legend(loc='lower right')
plt.title('VGG Training: Feature Extraction vs. Fine-Tuning')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

# Plot the combined loss
plt.figure(figsize=(10, 6))
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.axvline(initial_epoch - 1, color='red', linestyle='--', label='Start Fine-Tuning') # Add a line
plt.legend(loc='upper right')
plt.title('VGG Training: Feature Extraction vs. Fine-Tuning')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

##Cell 20: Analyze the Results (Fine-Tuning)
Look at the plots. You should see:

Epochs 0-4 (Feature Extraction): A rapid jump in accuracy as our new "head" learns.

Epoch 5 (The "Start Fine-Tuning" line): A small dip in accuracy. This is normal! The model is adjusting to the new, low learning rate and the "unfrozen" layers.

Epochs 5-15 (Fine-Tuning): A slow, steady climb as the model "fine-tunes" the VGG layers, squeezing out that last bit of performance to get an even higher final accuracy.

####We have successfully used Transfer Learning and Fine-Tuning!

##Cell 21: Save the Final Fine-Tuned Model

In [ ]:
# --- Define file name ---
final_vgg_model_path = "day4_vgg_fine_tuned_model.keras"

# --- Save the Final VGG Model ---
try:
    print(f"\nSaving Final VGG model to: {final_vgg_model_path} ...")
    model_vgg.save(final_vgg_model_path)
    print("Final VGG model saved successfully.")
except Exception as e:
    print(f"An error occurred while saving the final model: {e}")

##Cell 22: Final Predictions (VGG Model)
Let's run our best model, the fine-tuned VGG16, on a batch of test images. We should expect near-perfect results.

In [ ]:
# Get one batch of images and labels from the validation set
images_batch, labels_batch = next(iter(val_ds))

# Make predictions on this batch using the FINAL VGG model
print("Making predictions with model_vgg on a batch of validation images...")
predictions_batch = model_vgg.predict(images_batch)

# Get the predicted class indices
predicted_indices = np.argmax(predictions_batch, axis=1)
# Get the true class indices
true_indices = np.argmax(labels_batch.numpy(), axis=1)

# --- Plot the results ---
plt.figure(figsize=(10, 10))
print("Plotting Final VGG prediction grid...")

# Plot the first 9 images in the batch
# Note: The images will look "weird" because they are still 0-255 floats
# but matplotlib will auto-scale them for display.
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images_batch[i].numpy().astype("uint8"))

    pred_label = class_names[predicted_indices[i]]
    true_label = class_names[true_indices[i]]

    if pred_label == true_label:
        color = 'green'
    else:
        color = 'red'

    plt.title(f"Pred: {pred_label}\nTrue: {true_label}", color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()

##Cell 23: Peeking Inside the "Mind" of the VGG16 Model
Let's do the same visualization we did for our simple CNN. We'll take one test image and see what it looks like as it passes through the powerful VGG16 model.

We will:

Grab a single test image (e.g., a 'rotten apple').

Show the "preprocessed" image (what VGG16 actually sees).

Show the feature maps from the pooling layers of each block to see how the image becomes more abstract.

Show the final vector outputs from GlobalAveragePooling2D and the final prediction.

###Cell: 23a set-up


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Model

# --- 1. Get a single image to test ---
# Let's get one batch from the validation set
images_batch, labels_batch = next(iter(val_ds))

# Let's find a 'rottenapples' (index 3) to make it interesting
try:
    img_index = np.where(np.argmax(labels_batch.numpy(), axis=1) == 3)[0][0]
    img_to_visualize = images_batch[img_index]
    label_index = labels_batch.numpy()[img_index]

    # The model expects a "batch", so we add one dimension
    img_for_model = tf.expand_dims(img_to_visualize, axis=0)

    print(f"Loaded one image with shape: {img_for_model.shape}")
    print(f"True Label: {class_names[np.argmax(label_index)]}")

except IndexError:
    print("Could not find a 'rottenapples' in the first batch, using the first image instead.")
    img_to_visualize = images_batch[0]
    label_index = labels_batch.numpy()[0]
    img_for_model = tf.expand_dims(img_to_visualize, axis=0)


# Helper function to plot the feature maps
def plot_feature_maps(maps, title, n_features, grid_cols):
    grid_rows = int(np.ceil(n_features / grid_cols))
    plt.figure(figsize=(grid_cols * 2, grid_rows * 2))
    plt.suptitle(title, fontsize=16, y=1.02)

    # Squeeze the batch dimension
    maps = np.squeeze(maps)

    # Handle the case where maps are (height, width, channels)
    if len(maps.shape) == 3:
        # Plot only the first n_features
        for i in range(min(n_features, maps.shape[2])):
            plt.subplot(grid_rows, grid_cols, i + 1)
            plt.imshow(maps[:, :, i], cmap='viridis') # 'viridis' shows intensity well
            plt.title(f"Filter {i+1}")
            plt.axis('off')
    # Handle the case of a single map (like the rescaled image)
    elif len(maps.shape) == 2:
        plt.imshow(maps, cmap='viridis')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

###Step 1: Original vs. Preprocessed
The vgg16.preprocess_input function (in our Lambda layer) does more than just scale pixels to 0-1. It also changes the color channel order and subtracts the ImageNet mean pixel value.

This is why the "preprocessed" image looks so strange, but it's the exact format VGG16 was trained on.

In [ ]:
# --- 1. Get the Preprocessed Image ---
# We create a small model to just get the output of the preprocessing layer
try:
    preprocessing_layer = model_vgg.get_layer('vgg_preprocessing')
    viz_prep_model = Model(inputs=model_vgg.input, outputs=preprocessing_layer.output)
    preprocessed_img = viz_prep_model.predict(img_for_model)

    # --- 2. Plot them side-by-side ---
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.title(f"Original from Batch: {img_to_visualize.shape}")
    plt.imshow(img_to_visualize.numpy().astype("uint8"))
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title(f"Preprocessed (VGG's view): {preprocessed_img.shape}")
    # We need to clip the values back to 0-255 for display
    plt.imshow(np.clip(preprocessed_img[0], 0, 255).astype("uint8"))
    plt.axis('off')

    plt.show()

except Exception as e:
    print(f"Error during preprocessing visualization: {e}")

###Step 2: Visualizing the Conv/Pool Layers
Now, let's get the outputs from the pooling layer of each of the 5 VGG16 blocks.

Notice how the image starts as (224, 224) and is shrunk down at each step, becoming more and more abstract. The final maps (block5_pool) are just 7x7 but contain all the high-level concepts the model has found.

In [ ]:
# --- 1. Get the VGG Base Model ---
# We need to get the layers from the 'base_model_vgg' that is *inside* our main model
# (Note: your base model might be named 'vgg16' in its summary, use that if so)
try:
    base_model_layer = model_vgg.get_layer('vgg16') # This is the default name Keras gives it
except ValueError:
    # Fallback if you named the base model layer differently
    base_model_layer = base_model_vgg

# --- 2. List the layers we want to see ---
layer_names = [
    'block1_pool',
    'block2_pool',
    'block3_pool',
    'block4_pool',
    'block5_pool'
]
layer_outputs = [base_model_layer.get_layer(name).output for name in layer_names]

# --- 3. Create the visualization model ---
viz_conv_model = Model(inputs=base_model_layer.input, outputs=layer_outputs)

# --- 4. Get the activations ---
# We must feed the *preprocessed_img* from the previous step
conv_activations = viz_conv_model.predict(preprocessed_img)
print("Conv/Pool activations generated.")

In [ ]:
# Plot the maps!
# We'll just show the first 8 filters from each block to save space

print("--- Block 1 Pool (112x112) ---")
plot_feature_maps(conv_activations[0], "Block 1 Pool (112x112) - 8/64 Filters", n_features=8, grid_cols=4)

print("\n--- Block 2 Pool (56x56) ---")
plot_feature_maps(conv_activations[1], "Block 2 Pool (56x56) - 8/128 Filters", n_features=8, grid_cols=4)

print("\n--- Block 3 Pool (28x28) ---")
plot_feature_maps(conv_activations[2], "Block 3 Pool (28x28) - 8/256 Filters", n_features=8, grid_cols=4)

print("\n--- Block 4 Pool (14x14) ---")
plot_feature_maps(conv_activations[3], "Block 4 Pool (14x14) - 8/512 Filters", n_features=8, grid_cols=4)

print("\n--- Block 5 Pool (7x7) ---")
plot_feature_maps(conv_activations[4], "Block 5 Pool (7x7) - 8/512 Filters", n_features=8, grid_cols=4)

###Step 3: The Vector Part (From Features to Prediction)
This is where the image stops being an image and becomes a 1D vector.

GlobalAveragePooling2D: Takes the 512 7x7 feature maps from block5_pool and outputs a single vector of 512 numbers (the average of each map).

Dense (Hidden): This 512-feature vector is fed to our hidden Dense layer (128 neurons).

Dense (Output): The 128 neurons are fed to the final Dense layer (6 neurons), which gives us our final softmax prediction.

In [ ]:
# --- 1. List the "head" layers ---
layer_names = [
    'global_avg_pool',
    'dense_head',
    'output_layer'
]
layer_outputs = [model_vgg.get_layer(name).output for name in layer_names]

# --- 2. Create the "head" visualization model ---
viz_head_model = Model(inputs=model_vgg.input, outputs=layer_outputs)

# --- 3. Get the vector outputs ---
# We feed the *original* image, as this model includes the preprocessor
head_activations = viz_head_model.predict(img_for_model)

# --- 4. Print the outputs ---
print("--- Step 1: Global Average Pooling Output ---")
print(f"Shape: {head_activations[0].shape}  (1 batch, 512 features)")
print(f"First 10 values: {head_activations[0][0, :10]}")

print("\n--- Step 2: Dense Hidden Layer (128 Neurons) Output ---")
print(f"Shape: {head_activations[1].shape}")
print(f"First 10 values: {head_activations[1][0, :10]}")

print("\n--- Step 3: Dense Output Layer (6 Neurons) ---")
print(f"Shape: {head_activations[2].shape}")
print(f"Probabilities (softmax): \n{head_activations[2][0]}")

# Find the final prediction
final_prediction_index = np.argmax(head_activations[2][0])
final_prediction_label = class_names[final_prediction_index]
print("\n---")
print(f"FINAL PREDICTION: {final_prediction_label}")

##Day 4 Conclusion
You have now seen the full spectrum of image classification!

MLP (Failed): We proved that a simple Dense network is the wrong tool for image data. It overfits because it can't understand 2D space.

Basic CNN (Good): We proved that Conv2D layers are the right tool. This model learned to see features and performed well (~97%).

Pro CNN (Better): We made our CNN deeper and added BatchNormalization and GlobalAveragePooling2D. This reduced overfitting and created a very robust model (~98-99%).

Transfer Learning (Best): We scrapped our own model and used VGG16, a model pre-trained on millions of images. We achieved the highest accuracy (~99%+) in the fewest epochs.

###Takeaway: Never train from scratch if you don't have to! Transfer Learning is the fastest, most powerful way to get state-of-the-art results.

it's Smart, Not Hard: Instead of training a model from scratch (like in Day 2 & 3), you've used Transfer Learning. You loaded VGG16, a massive model pre-trained by experts on millions of images (ImageNet). This means it already knows how to see edges, shapes, colors, and textures.

Highly Efficient: Your notebook cleverly "freezes" the 14.7 million parameters of the VGG16 base and only trains a tiny new "head" (about 66,000 parameters) that you added. This is why it achieved very high accuracy in just 5 epochs.

Pro-Level Technique: You then used Fine-Tuning by "unfreezing" the top layers of VGG16 and re-training them with a very low learning rate. This "tunes" the pre-trained features to be even better at your specific task of identifying fruit.

You have now seen the full spectrum of image classification!

MLP (Failed): We proved that a simple Dense network is the wrong tool for image data.

Basic CNN (Good): We proved that Conv2D layers are the right tool and got ~97% accuracy.

Pro CNN (Better): We built a deeper, regularized model to get ~99% accuracy.

Transfer Learning (Best): We "borrowed the brain" of VGG16, which already knew how to see. This gave us the highest accuracy (~99%+) in the fewest epochs.